In [1]:
# Import findspark and initialize. 
import findspark
findspark.init()

In [2]:
# Import packages
from pyspark.sql import SparkSession
import time

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

In [3]:
# 1. Read in the AWS S3 bucket into a DataFrame.
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"



In [4]:
# 2. Create a temporary view of the DataFrame.
import pandas as pd

# Load the CSV file using Pandas
pdf = pd.read_csv(url)

# Show the first few rows of the Pandas DataFrame
print(pdf.head())

# Convert Pandas DataFrame to Spark DataFrame
df = spark.createDataFrame(pdf)

# Show the first few rows of the Spark DataFrame
df.show()

# Create a temporary view of the DataFrame
df.createOrReplaceTempView("home_sales")



                                     id        date  date_built   price  \
0  f8a53099-ba1c-47d6-9c31-7398aa8f6089  2022-04-08        2016  936923   
1  7530a2d8-1ae3-4517-9f4a-befe060c4353  2021-06-13        2013  379628   
2  43de979c-0bf0-4c9f-85ef-96dc27b258d5  2019-04-12        2014  417866   
3  b672c137-b88c-48bf-9f18-d0a4ac62fb8b  2019-10-16        2016  239895   
4  e0726d4d-d595-4074-8283-4139a54d0d63  2022-01-08        2017  424418   

   bedrooms  bathrooms  sqft_living  sqft_lot  floors  waterfront  view  
0         4          3         3167     11733       2           1    76  
1         2          2         2235     14384       1           0    23  
2         2          2         2127     10575       2           0     0  
3         2          2         1631     11149       2           0     0  
4         3          2         2249     13878       2           0     4  
+--------------------+----------+----------+------+--------+---------+-----------+--------+------+-------

In [5]:
# 3. What is the average price for a four bedroom house sold per year, rounded to two decimal places?
query1 = """
    SELECT year(date) AS year, ROUND(AVG(price), 2) AS avg_price
    FROM home_sales
    WHERE bedrooms = 4
    GROUP BY year(date)
    ORDER BY year(date)
"""
result1 = spark.sql(query1)
result1.show()



+----+---------+
|year|avg_price|
+----+---------+
|2019| 300263.7|
|2020|298353.78|
|2021|301819.44|
|2022|296363.88|
+----+---------+



In [13]:
# 4. What is the average price of a home for each year the home was built,
# that have 3 bedrooms and 3 bathrooms, rounded to two decimal places
# Print the schema of the DataFrame to verify column names
df.printSchema()

# Adjust the query based on the correct column names
query2 = """
    SELECT yr_built AS date_built, ROUND(AVG(price), 2) AS avg_price
    FROM home_sales
    WHERE bedrooms = 3 AND bathrooms = 3
    GROUP BY yr_built
    ORDER BY yr_built
"""

result2 = spark.sql(query2)
result2.show()




root
 |-- id: string (nullable = true)
 |-- date: string (nullable = true)
 |-- date_built: long (nullable = true)
 |-- price: long (nullable = true)
 |-- bedrooms: long (nullable = true)
 |-- bathrooms: long (nullable = true)
 |-- sqft_living: long (nullable = true)
 |-- sqft_lot: long (nullable = true)
 |-- floors: long (nullable = true)
 |-- waterfront: long (nullable = true)
 |-- view: long (nullable = true)



AnalysisException: [UNRESOLVED_COLUMN.WITH_SUGGESTION] A column or function parameter with name `yr_built` cannot be resolved. Did you mean one of the following? [`date_built`, `id`, `date`, `floors`, `price`].; line 2 pos 11;
'Sort ['yr_built ASC NULLS FIRST], true
+- 'Aggregate ['yr_built], ['yr_built AS date_built#506, round(avg(price#3L), 2) AS avg_price#507]
   +- Filter ((bedrooms#4L = cast(3 as bigint)) AND (bathrooms#5L = cast(3 as bigint)))
      +- SubqueryAlias home_sales
         +- View (`home_sales`, [id#0,date#1,date_built#2L,price#3L,bedrooms#4L,bathrooms#5L,sqft_living#6L,sqft_lot#7L,floors#8L,waterfront#9L,view#10L])
            +- LogicalRDD [id#0, date#1, date_built#2L, price#3L, bedrooms#4L, bathrooms#5L, sqft_living#6L, sqft_lot#7L, floors#8L, waterfront#9L, view#10L], false


In [8]:
# 5. What is the average price of a home for each year the home was built,
# that have 3 bedrooms, 3 bathrooms, with two floors,
# and are greater than or equal to 2,000 square feet, rounded to two decimal places?
query3 = """
    SELECT yr_built AS date_built, ROUND(AVG(price), 2) AS avg_price
    FROM home_sales
    WHERE bedrooms = 3 AND bathrooms = 3 AND floors = 2 AND sqft_living >= 2000
    GROUP BY yr_built
    ORDER BY yr_built
"""
result3 = spark.sql(query3)
result3.show()



AnalysisException: [UNRESOLVED_COLUMN.WITH_SUGGESTION] A column or function parameter with name `yr_built` cannot be resolved. Did you mean one of the following? [`date_built`, `id`, `date`, `floors`, `price`].; line 2 pos 11;
'Sort ['yr_built ASC NULLS FIRST], true
+- 'Aggregate ['yr_built], ['yr_built AS date_built#94, round(avg(price#3L), 2) AS avg_price#95]
   +- Filter (((bedrooms#4L = cast(3 as bigint)) AND (bathrooms#5L = cast(3 as bigint))) AND ((floors#8L = cast(2 as bigint)) AND (sqft_living#6L >= cast(2000 as bigint))))
      +- SubqueryAlias home_sales
         +- View (`home_sales`, [id#0,date#1,date_built#2L,price#3L,bedrooms#4L,bathrooms#5L,sqft_living#6L,sqft_lot#7L,floors#8L,waterfront#9L,view#10L])
            +- LogicalRDD [id#0, date#1, date_built#2L, price#3L, bedrooms#4L, bathrooms#5L, sqft_living#6L, sqft_lot#7L, floors#8L, waterfront#9L, view#10L], false


In [9]:
# 6. What is the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000? Order by descending view rating. 
# Although this is a small dataset, determine the run time for this query.

import time

start_time = time.time()

query4 = """
    SELECT view, ROUND(AVG(price), 2) AS avg_price
    FROM home_sales
    GROUP BY view
    HAVING AVG(price) >= 350000
    ORDER BY avg_price DESC
"""
result4 = spark.sql(query4)
result4.show()

end_time = time.time()
uncached_runtime = end_time - start_time
print(f"Uncached query runtime: {uncached_runtime:.2f} seconds")


+----+----------+
|view| avg_price|
+----+----------+
|  91|1137372.73|
|  97|1129040.15|
|  84|1117233.13|
|  75|1114042.94|
|  89|1107839.15|
|  78|1080649.37|
|  77|1076205.56|
|  87| 1072285.2|
|  86|1070444.25|
|  82| 1063498.0|
|  90|1062654.16|
|  99|1061201.42|
|  76|1058802.78|
|  85|1056336.74|
|  95| 1054325.6|
|  98|1053739.33|
|  81|1053472.79|
|  83|1033965.93|
|  94| 1033536.2|
|  88|1031719.35|
+----+----------+
only showing top 20 rows

Uncached query runtime: 9.97 seconds


In [10]:
# 7. Cache the the temporary table home_sales.
# Cache the table
spark.catalog.cacheTable("home_sales")

# Check if table is cached
is_cached = spark.catalog.isCached("home_sales")
print(f"Is 'home_sales' cached? {is_cached}")


Is 'home_sales' cached? True


In [ ]:
# 8. Check if the table is cached.
#see above please
spark.catalog.isCached('home_sales')

In [11]:
# 9. Using the cached data, run the last query above, that calculates 
# the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000. 
# Determine the runtime and compare it to the uncached runtime.

start_time = time.time()

result4_cached = spark.sql(query4)
result4_cached.show()

end_time = time.time()
cached_runtime = end_time - start_time
print(f"Cached query runtime: {cached_runtime:.2f} seconds")



+----+----------+
|view| avg_price|
+----+----------+
|  91|1137372.73|
|  97|1129040.15|
|  84|1117233.13|
|  75|1114042.94|
|  89|1107839.15|
|  78|1080649.37|
|  77|1076205.56|
|  87| 1072285.2|
|  86|1070444.25|
|  82| 1063498.0|
|  90|1062654.16|
|  99|1061201.42|
|  76|1058802.78|
|  85|1056336.74|
|  95| 1054325.6|
|  98|1053739.33|
|  81|1053472.79|
|  83|1033965.93|
|  94| 1033536.2|
|  88|1031719.35|
+----+----------+
only showing top 20 rows

Cached query runtime: 11.07 seconds


In [10]:
# 10. Partition by the "date_built" field on the formatted parquet home sales data 
# Write the data to parquet format partitioned by date_built
df.write.partitionBy("date_built").parquet("/mnt/data/home_sales_parquet", mode="overwrite")



NameError: name 'df' is not defined

In [11]:
# 11. Read the formatted parquet data.
# Read the parquet data into a new DataFrame and create a temporary table
df_parquet = spark.read.parquet("/mnt/data/home_sales_parquet")
df_parquet.createOrReplaceTempView("home_sales_parquet")



NameError: name 'spark' is not defined

In [ ]:
# 12. Create a temporary table for the parquet data.
# Read the parquet data into a new DataFrame and create a temporary table
#see above


In [12]:
# 13. Using the parquet DataFrame, run the last query above, that calculates 
# the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000. 
# Determine the runtime and compare it to the cached runtime.
# Run the query on the parquet data
start_time_parquet = time.time()
result_4_parquet = spark.sql(query_4)
result_4_parquet.show()
end_time_parquet = time.time()

runtime_parquet = end_time_parquet - start_time_parquet
print(f"Runtime (parquet): {runtime_parquet} seconds")


NameError: name 'time' is not defined

In [13]:
# 14. Uncache the home_sales temporary table.
# Uncache the temporary table
spark.catalog.uncacheTable("home_sales")


NameError: name 'spark' is not defined

In [14]:
# 15. Check if the home_sales is no longer cached
# Verify if the table is uncached
is_cached_after = spark.catalog.isCached("home_sales")
print(f"Is home_sales table cached after uncache? {is_cached_after}")



NameError: name 'spark' is not defined